In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import sounddevice as sd
import scipy
import pickle
import librosa
import pyloudnorm as pyln
%matplotlib qt5

import warnings
warnings.filterwarnings("ignore")


In [1]:
# import lpc_synth notebook
%run lpc_synth.ipynb
fs = 44100

In [ ]:
with open('lpc_coeffs/vowelsf3', 'rb') as file:
    lpc_coeffs = pickle.load(file)

# noise = volume fluctuations
# 0.2 noise is a nice value
def gen_vowel(v, f0, T, slide = None, tremolo=0.25, air = 0.012, slide2 = None):
    lpc = lpc_coeffs[v]
    if v in vowels:
        lpc_air = lpc_coeffs[f"{v}_air"]
        y_air = noise_filter(lpc_air, T, magnitude=air)

    # TODO: change coeffs based on f0
    if slide is None:
        slide = FormantSlide(formant_thresh=1000, low_slide_start=0.9, high_slide_start=0.9)

    y = glottal_filter(lpc, f0, T, noise=0.012, slide=slide, slide2=slide2)

    brown_noise = gen_brown_noise(len(y))
    # upscale the noise so it is slower
    brown_noise = scipy.signal.resample(brown_noise, 3 * len(y))[0:len(y)] 
    brown_noise = brown_noise * tremolo + 1

    if v in vowels:
        y += y_air
    y *= brown_noise

    meter = pyln.Meter(fs, block_size=0.05)
    loudness = meter.integrated_loudness(y)
    y = pyln.normalize.loudness(y, loudness, -12.0)

    return y

In [ ]:
# y = gen_vowel('u', f0=174, T=2.0)
# play_audio(y)